In [1]:
%cd /home/yz979/code/kaggle-perturbation
!ls data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import scanpy as sc

/storage/home/yz979/code/kaggle-perturbation
adata_obs_meta.csv   de_train.parquet	     sample_submission.csv
adata_train.h5ad     grn		     submission_1.csv
adata_train.parquet  id_map.csv		     submission_2.csv
cpa_model	     id_map.h5ad	     submission_3.csv
cpa_model_2	     id_map.parquet	     submission_4.csv
cpa_model_adata      multiome_obs_meta.csv   unzip.py
de_train_genes.txt   multiome_train.parquet
de_train.h5ad	     multiome_var_meta.csv


## Explore ``adata_train.parquet``

In [11]:
adata_train = pd.read_parquet('data/adata_train.parquet')

In [55]:
unmelt = adata_train.pivot_table(index="obs_id", columns="gene", values="normalized_count").fillna(0)

/tmp/ipykernel_263081/2280732669.py:1: PerformanceWarning: The following operation may generate 5103112950 cells in the resulting pandas object.
  unmelt = adata_train.pivot_table(index="obs_id", columns="gene", values="normalized_count").fillna(0)


In [56]:
adata_obs_meta = pd.read_csv('data/adata_obs_meta.csv', index_col=0)

In [59]:
adata = anndata.AnnData(X=unmelt.values, obs=adata_obs_meta)
adata.write_h5ad('data/adata_train.h5ad')

In [60]:
adata

AnnData object with n_obs × n_vars = 240090 × 21255
    obs: 'library_id', 'plate_name', 'well', 'row', 'col', 'cell_id', 'donor_id', 'cell_type', 'sm_lincs_id', 'sm_name', 'SMILES', 'dose_uM', 'timepoint_hr', 'control'

In [34]:
adata_obs_meta = pd.read_csv('data/adata_obs_meta.csv')
len(adata_obs_meta['sm_name'].unique().tolist())

147

In [50]:
adata_obs_meta.head()

,obs_id,library_id,plate_name,well,row,col,cell_id,donor_id,cell_type,sm_lincs_id,sm_name,SMILES,dose_uM,timepoint_hr,control
0,000006a87ba75b72,library_4,plate_4,F7,F,7,PBMC,donor_2,T cells CD4+,LSM-4944,MLN 2238,CC(C)C[C@H](NC(=O)CNC(=O)c1cc(Cl)ccc1Cl)B(O)O,1.0,24,False
1,0000233976e3cd37,library_0,plate_3,D4,D,4,PBMC,donor_1,T cells CD4+,LSM-46203,BMS-265246,CCCCOc1c(C(=O)c2c(F)cc(C)cc2F)cnc2[nH]ncc12,1.0,24,False
2,0001533c5e876362,library_2,plate_0,B11,B,11,PBMC,donor_0,T regulatory cells,LSM-45663,Resminostat,CN(C)Cc1ccc(S(=O)(=O)n2ccc(/C=C/C(=O)NO)c2)cc1,1.0,24,False
3,00022f989630d14b,library_35,plate_2,E6,E,6,PBMC,donor_0,T cells CD4+,LSM-43216,FK 866,O=C(/C=C/c1cccnc1)NCCCCC1CCN(C(=O)c2ccccc2)CC1,1.0,24,False
4,0002560bd38ce03e,library_22,plate_4,B6,B,6,PBMC,donor_2,T cells CD4+,LSM-1099,Nilotinib,Cc1cn(-c2cc(NC(=O)c3ccc(C)c(Nc4nccc(-c5cccnc5)...,1.0,24,False


In [3]:
de_train = pd.read_parquet('data/de_train.parquet')
X = de_train.iloc[:, 5:]

# de_train_ = anndata.AnnData(X=X, obs=de_train.iloc[:, :5])

# # save adata
# de_train_.write('data/de_train.h5ad')

In [ ]:
genes = de_train_.var_names

In [35]:
len(de_train['sm_name'].unique().tolist())

146

In [2]:
de_train

# find 1:1 mapping of sm_name and sm_lincs_id

sm_name = de_train.sm_name.unique()
sm_lincs_id = de_train.sm_lincs_id.unique()


def find_sm_lincs_id(sm_name):
    return de_train[de_train.sm_name == sm_name].sm_lincs_id.unique()[0]


sm_lincs_id_map = {sm_name: find_sm_lincs_id(sm_name) for sm_name in sm_name}

NameError: name 'de_train' is not defined

In [49]:
multiome_train = pd.read_parquet('data/multiome_train.parquet')

In [39]:
multiome_train.head()

(216251368, 4)

In [40]:
multiome_var_meta = pd.read_csv('data/multiome_var_meta.csv')

In [43]:
multiome_var_meta

,location,gene_id,feature_type,genome,interval
0,A1BG,ENSG00000121410,Gene Expression,GRCh38,chr19:58353491-58353492
1,A1BG-AS1,ENSG00000268895,Gene Expression,GRCh38,chr19:58347750-58351970
2,A2M,ENSG00000175899,Gene Expression,GRCh38,chr12:9116156-9116157
3,A2M-AS1,ENSG00000245105,Gene Expression,GRCh38,chr12:9065162-9065177
4,A2ML1,ENSG00000166535,Gene Expression,GRCh38,chr12:8822620-8845004
...,...,...,...,...,...
158200,chrY:7765105-7765991,chrY:7765105-7765991,Peaks,GRCh38,chrY:7765105-7765991
158201,chrY:7814158-7815060,chrY:7814158-7815060,Peaks,GRCh38,chrY:7814158-7815060
158202,chrY:7818681-7819599,chrY:7818681-7819599,Peaks,GRCh38,chrY:7818681-7819599
158203,chrY:8535565-8536421,chrY:8535565-8536421,Peaks,GRCh38,chrY:8535565-8536421


In [4]:
id_map = pd.read_csv('data/id_map.csv', index_col='id')
# id_map to parquet
id_map.to_parquet('data/id_map.parquet')

# id_map to anndata
X_ = np.zeros((id_map.shape[0], X.shape[1]))
X_ = pd.DataFrame(X_, index=id_map.index, columns=X.columns)

# false
id_map['control'] = np.zeros(id_map.shape[0], dtype=bool)

# all row of X_ is the average of all rows in X that has the same cell type
for i, row in id_map.iterrows():
    cell_type = row['cell_type']
    X_.loc[i] = X[de_train['cell_type'] == cell_type].mean()

adata_ = anndata.AnnData(X=X_, obs=id_map)
adata_.write('data/id_map.h5ad')

/home/yz979/miniconda3/envs/kaggle/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [5]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251,251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252,252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253,253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
submission_1 = pd.read_csv('data/submission_1.csv')
submission_1

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,-0.012740,0.001592,0.004088,0.019184,0.008488,-0.001554,0.001697,0.002251,-0.000972,...,-0.011493,-0.007042,-0.011486,-0.003998,0.001426,-0.007347,0.002272,0.002001,0.003667,0.012909
1,1,-0.009417,-0.000347,0.020844,-0.003002,-0.010366,-0.014830,0.003348,0.010439,-0.001598,...,-0.017710,0.004704,0.009680,-0.004831,0.003418,-0.001047,0.010184,0.010008,-0.001782,0.008418
2,2,-0.019360,-0.001013,-0.008138,-0.009563,-0.012286,-0.004634,-0.006597,0.004652,-0.009238,...,-0.011224,-0.009196,-0.014939,-0.002523,0.004310,-0.012358,0.002696,0.008391,0.007799,0.008638
3,3,-0.002294,-0.005512,0.011484,0.017709,0.005581,-0.002822,-0.000882,-0.002886,0.008856,...,0.001323,-0.005732,-0.013957,-0.028611,0.006617,0.000935,0.003347,0.003180,0.000077,0.015722
4,4,-0.002396,-0.000336,0.041019,0.035460,0.000649,0.014993,-0.002897,0.003944,0.001670,...,-0.010314,0.001272,-0.004189,-0.009112,0.007048,0.002809,0.009853,0.008642,0.003202,0.012314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,-0.000935,0.001697,0.028642,0.008283,0.012555,-0.003017,0.003531,-0.000465,0.007392,...,-0.000031,-0.003096,-0.008941,-0.006887,0.003971,0.007714,0.000347,-0.000716,0.007249,0.013979
251,251,-0.007878,0.002978,0.029206,0.012550,0.002944,0.018456,-0.004860,-0.006191,0.000878,...,-0.006118,-0.007269,-0.005962,0.002898,0.002091,0.001728,-0.002615,-0.006562,0.011390,0.002218
252,252,-0.006622,0.005312,-0.024126,-0.015099,-0.009781,0.030170,0.004736,0.004407,0.004860,...,0.002599,-0.000397,-0.010291,0.003203,0.002435,-0.006053,-0.008527,-0.000888,0.007700,0.015914
253,253,-0.010787,0.001259,-0.007673,0.015963,0.015887,0.012076,-0.006711,-0.000344,0.002433,...,0.001010,-0.006290,-0.004898,0.001917,0.012009,-0.003579,0.002891,-0.003620,0.002630,0.001677


In [6]:
%cd /home/yz979/code/kaggle-perturbation
!kaggle competitions submit -c open-problems-single-cell-perturbations -f data/submission_6.csv -m "submit"

/storage/home/yz979/code/kaggle-perturbation


100%|██████████████████████████████████████| 49.8M/49.8M [00:01<00:00, 38.6MB/s]
400 - Bad Request - Submission not allowed:  Your team has used its daily Submission allowance (5) today, please try again tomorrow UTC (11 hours from now).
